In [3]:
import pandas as pd
from pandas import Timestamp

In [4]:

df = pd.read_excel("D:/DATA SKRIPSI/Kontrak-Sewa-Chelsie-250422.xlsx")

df['LeaseStartDate'] = pd.to_datetime(df['LeaseStartDate'], errors='coerce')
df['LeaseEndDate']   = pd.to_datetime(df['LeaseEndDate'], errors='coerce')

start_date = Timestamp("2015-01-01")
end_date   = pd.Timestamp.today() 

mask = df['LeaseStartDate'].between(start_date, end_date) & \
       df['LeaseEndDate'].between(start_date, end_date)

df_new = df[mask]



In [18]:
df_new = pd.read_excel("kontrak-sewa.xlsx")

In [27]:
df_new.iloc[:,10:20].head

<bound method NDFrame.head of        BuildingArea  CuryUnitPrice          UnitID
0               1.0   2.142858e+04  0TG00K04000001
1               1.0   2.678572e+04  0TG00K04000001
2               1.0   6.818182e+04  0PEATRLG000009
3               1.0   6.818182e+04  0PEATRLG000009
4               1.0   6.818182e+04  0PEATRLG000008
...             ...            ...             ...
9819            1.0   1.147500e+08  0PEATRLG000061
10687           1.0   5.000000e+07  0PWP0101000001
10688           1.0   5.000000e+07  0PWP0101000001
11367           1.0   5.000000e+07  0PWP0101000001
11369           1.0   1.250000e+08  0TGATN01000001

[6687 rows x 3 columns]>

In [28]:
df_new["CuryUnitPrice"]

0        2.142858e+04
1        2.678572e+04
2        6.818182e+04
3        6.818182e+04
4        6.818182e+04
             ...     
9819     1.147500e+08
10687    5.000000e+07
10688    5.000000e+07
11367    5.000000e+07
11369    1.250000e+08
Name: CuryUnitPrice, Length: 6687, dtype: float64

### Adding new features

In [29]:
print(df_new.isnull().sum())

MCNbr             0
CustID            0
CustName          0
LeaseStartDate    0
LeaseEndDate      0
ContractType      0
ContractPeriod    0
BusinessType      0
Status            0
TranCodeID        0
BuildingArea      0
CuryUnitPrice     0
UnitID            0
dtype: int64


In [6]:
df_new = df_new.copy()

for col in df_new.columns:
    if col != "CustID":
        df_new.loc[:, col] = df_new.groupby("CustID")[col].transform(
            lambda x: x.fillna(x.mode()[0] if not x.mode().empty else x)
        )


In [30]:
# Convert ke datetime
df_new["LeaseStartDate"] = pd.to_datetime(df_new["LeaseStartDate"])
df_new["LeaseEndDate"]   = pd.to_datetime(df_new["LeaseEndDate"])

# Convert ke datetime
df_new["LeaseStartDate"] = pd.to_datetime(df_new["LeaseStartDate"])
df_new["LeaseEndDate"]   = pd.to_datetime(df_new["LeaseEndDate"])

# Ekstraksi variabel baru
df_new["LeaseYearStart"]   = df_new["LeaseStartDate"].dt.year
df_new["LeaseMonthStart"]  = df_new["LeaseStartDate"].dt.month
df_new["LeaseDayStart"]    = df_new["LeaseStartDate"].dt.day

df_new["LeaseYearEnd"]     = df_new["LeaseEndDate"].dt.year
df_new["LeaseMonthEnd"]    = df_new["LeaseEndDate"].dt.month
df_new["LeaseDayEnd"]      = df_new["LeaseEndDate"].dt.day

df_new["LeaseDurationDays"]   = (df_new["LeaseEndDate"] - df_new["LeaseStartDate"]).dt.days
df_new["LeaseDurationMonths"] = ((df_new["LeaseEndDate"].dt.year - df_new["LeaseStartDate"].dt.year) * 12 + 
                              (df_new["LeaseEndDate"].dt.month - df_new["LeaseStartDate"].dt.month))

df_new = df_new.drop(columns=["LeaseStartDate", "LeaseEndDate"])
print(df_new.iloc[:, 13:])

       LeaseDayStart  LeaseYearEnd  LeaseMonthEnd  LeaseDayEnd  \
0                 28          2016              5            1   
1                 29          2016              3           27   
2                 13          2022             10           23   
3                 13          2022             10           23   
4                 13          2022             10           23   
...              ...           ...            ...          ...   
9819               1          2015              6           21   
10687             11          2016             10           10   
10688             11          2016              7           10   
11367             11          2016              5           10   
11369             10          2020              9            9   

       LeaseDurationDays  LeaseDurationMonths  
0                     34                    2  
1                     27                    1  
2                     10                    0  
3              

C:\Users\HP\AppData\Local\Temp\ipykernel_17220\265834879.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["LeaseStartDate"] = pd.to_datetime(df_new["LeaseStartDate"])
C:\Users\HP\AppData\Local\Temp\ipykernel_17220\265834879.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["LeaseEndDate"]   = pd.to_datetime(df_new["LeaseEndDate"])
C:\Users\HP\AppData\Local\Temp\ipykernel_17220\265834879.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

In [ ]:
"""
# Buat list label (a, b, c, ... z, aa, ab, ...)
def generate_labels(n):
    labels = []
    for i in range(n):
        s = ""
        j = i
        while True:
            s = string.ascii_lowercase[j % 26] + s
            j = j // 26 - 1
            if j < 0:
                break
        labels.append(s)
    return labels

# Fungsi untuk memberi subunitID per grup
def assign_subunit_ids(group):
    labels = generate_labels(len(group))
    group["subunitID"] = labels
    return group

# Terapkan ke setiap grup berdasarkan MCNbr dan custID
df_new = df_new.groupby(["MCNbr", "custID"], group_keys=False).apply(assign_subunit_ids)
"""

In [ ]:

df_new["n_subunit"] = df_new.groupby(["MCNbr", "CustID"])["UnitID"].transform("count")

In [ ]:
# memisahkan bagian" dari unitID sesuai posisi

df_new["Building"] = df_new["UnitID"].str[0:3]   
df_new["UnitArea"] = df_new["UnitID"].str[3:6]   
df_new["UnitFloor"] = df_new["UnitID"].str[6:8]
df_new["UnitNum"] = df_new["UnitID"].str[8:]    

print(df_new)

       MCNbr           CustID            CustName    ContractType  \
0      43176  I0078            PT I0078            Casual Leasing   
1      43175  N0122            PT N0122            Casual Leasing   
2      43173  N0122            PT N0122            Casual Leasing   
3      43162  J0047            PT J0047            Casual Leasing   
6      43149  P0214            PT P0214            Casual Leasing   
...      ...              ...                 ...             ...   
9819   12136  I0034            PT I0034            Casual Leasing   
10687  10300  M0014            PT M0014                   Leasing   
10688  10299  M0014            PT M0014                   Leasing   
11367   8257  M0014            PT M0014                   Leasing   
11369   8254  M0014            PT M0014                   Leasing   

      ContractPeriod  BusinessType Status TranCodeID  BuildingArea  \
0              Daily          99.0      H        LSA          1.00   
1              Daily          9

In [5]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6689 entries, 0 to 11369
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   MCNbr           6689 non-null   int64         
 1   CustID          6689 non-null   object        
 2   CustName        6689 non-null   object        
 3   LeaseStartDate  6689 non-null   datetime64[ns]
 4   LeaseEndDate    6689 non-null   datetime64[ns]
 5   ContractType    6689 non-null   object        
 6   ContractPeriod  6689 non-null   object        
 7   BusinessType    6689 non-null   float64       
 8   Status          6689 non-null   object        
 9   TranCodeID      6689 non-null   object        
 10  BuildingArea    6689 non-null   float64       
 11  CuryUnitPrice   6689 non-null   float64       
 12  UnitID          6689 non-null   object        
dtypes: datetime64[ns](2), float64(3), int64(1), object(7)
memory usage: 731.6+ KB


In [32]:
df_new = df_new.astype({
    'ContractType': 'string',
    'ContractPeriod': 'string',
    'TranCodeID': 'string'
})

In [33]:
df_new = df_new.rename(columns={'TranCodeID': 'TranCode'})


In [ ]:
df_new = df_new.drop(columns=["Status", "CustName", "UnitID"])

print("\nSetelah kolom dihapus:")
print(df_new)


Setelah kolom dihapus:
       MCNbr           CustID    ContractType ContractPeriod  BusinessType  \
0      19880  G0076            Casual Leasing          Daily          99.0   
1      19661  G0076            Casual Leasing          Daily          99.0   
2      36661  J0159            Casual Leasing          Daily          99.0   
3      36594  J0159            Casual Leasing          Daily          99.0   
4      36593  I0249            Casual Leasing          Daily          99.0   
...      ...              ...             ...            ...           ...   
9819   17409  M0014            Casual Leasing          Daily          99.0   
10687  20575  P0104            Casual Leasing        Monthly          99.0   
10688  20149  P0104            Casual Leasing        Monthly          99.0   
11367  19893  P0104            Casual Leasing        Monthly          99.0   
11369  18584  T0006                   Leasing        Monthly          99.0   

      TranCode  BuildingArea  CuryUnitP

In [35]:
df_new = df_new.drop(columns=["MCNbr", "CustID"])

print("\nSetelah kolom dihapus:")
print(df_new)


Setelah kolom dihapus:
         ContractType ContractPeriod  BusinessType TranCode  BuildingArea  \
0      Casual Leasing          Daily          99.0      LSC           1.0   
1      Casual Leasing          Daily          99.0      LSC           1.0   
2      Casual Leasing          Daily          99.0      LAC           1.0   
3      Casual Leasing          Daily          99.0      LAC           1.0   
4      Casual Leasing          Daily          99.0      LAC           1.0   
...               ...            ...           ...      ...           ...   
9819   Casual Leasing          Daily          99.0      LAC           1.0   
10687  Casual Leasing        Monthly          99.0      LSA           1.0   
10688  Casual Leasing        Monthly          99.0      LSA           1.0   
11367  Casual Leasing        Monthly          99.0      LSA           1.0   
11369         Leasing        Monthly          99.0      LSN           1.0   

       CuryUnitPrice          UnitID  LeaseYearStar

In [36]:
COL_NAME = "ContractPeriod"   

# Bersihkan nilai agar konsisten (hilangkan spasi, case-insensitive)
tmp = (
    pd.Series(pd.NA if COL_NAME not in df_new.columns else df_new[COL_NAME])
      .astype("string")
      .str.strip()
      .str.lower()
)

# Buat dua subset
df_monthly = df_new[tmp == "monthly"].copy()
df_daily   = df_new[tmp == "daily"].copy()

# Tulis ke Excel: Sheet1 = Monthly, Sheet2 = Daily
out_path = "output_split_contract_period_raw.xlsx"
with pd.ExcelWriter(out_path, engine="xlsxwriter") as writer:
    df_monthly.to_excel(writer, index=False, sheet_name="Monthly")  # Sheet 1
    df_daily.to_excel(writer,   index=False, sheet_name="Daily")    # Sheet 2

print(f"File tersimpan: {out_path}")

File tersimpan: output_split_contract_period_raw.xlsx


In [ ]:
df_new.to_excel("kontrak-sewa-not-encoded.xlsx", index=False)

## Encode categorical column

In [55]:
df_new

,ContractType,ContractPeriod,BusinessType,TranCode,BuildingArea,CuryUnitPrice,LeaseYearStart,LeaseMonthStart,LeaseDayStart,LeaseYearEnd,LeaseMonthEnd,LeaseDayEnd,LeaseDurationDays,LeaseDurationMonths,n_subunit,Building,UnitArea,UnitFloor,UnitNum
0,Casual Leasing,Daily,99.0,LSA,1.00,9.428571e+05,2025,5,12,2025,6,1,20,1,1,0PE,P01,01,000001
1,Casual Leasing,Daily,99.0,LAC,1.00,4.142857e+06,2025,5,19,2025,5,25,6,0,1,0PE,ATR,LG,000051
2,Casual Leasing,Daily,99.0,LAC,1.00,4.142857e+06,2025,5,5,2025,5,18,13,0,1,0PE,ATR,LG,000051
3,Casual Leasing,Daily,99.0,LCC,1.00,1.666667e+07,2025,5,16,2025,5,18,2,0,1,0PC,0CV,06,000051
6,Casual Leasing,Daily,99.0,LAC,1.00,7.200000e+05,2025,5,16,2025,5,25,9,0,1,0PC,ATR,LG,000006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9819,Casual Leasing,Daily,99.0,LSA,1.00,4.741964e+06,2017,6,12,2017,6,18,6,0,1,0PC,P01,LG,000068
10687,Leasing,Monthly,99.0,LSN,362.97,6.189752e+04,2016,1,5,2017,1,4,365,12,1,0PE,000,UG,000019
10688,Leasing,Monthly,99.0,LSN,362.97,5.731252e+04,2015,1,5,2016,1,4,364,12,1,0PE,000,UG,000019
11367,Leasing,Monthly,99.0,LSN,6771.65,6.189752e+04,2016,1,5,2017,1,4,365,12,1,0PE,000,UG,000020


In [ ]:
# encode categorical column (one hot)
df_encoded = pd.get_dummies(df_new, columns=["ContractType", "ContractPeriod", "TranCode", "Building", "UnitArea", "UnitFloor", "BusinessType"])

print("\nSetelah One Hot Encoding:")
print(df_encoded)


Setelah One Hot Encoding:
       BusinessType  BuildingArea  CuryUnitPrice  LeaseYearStart  \
0              99.0          1.00   9.428571e+05            2025   
1              99.0          1.00   4.142857e+06            2025   
2              99.0          1.00   4.142857e+06            2025   
3              99.0          1.00   1.666667e+07            2025   
6              99.0          1.00   7.200000e+05            2025   
...             ...           ...            ...             ...   
9819           99.0          1.00   4.741964e+06            2017   
10687          99.0        362.97   6.189752e+04            2016   
10688          99.0        362.97   5.731252e+04            2015   
11367          99.0       6771.65   6.189752e+04            2016   
11369          99.0       6771.65   5.731252e+04            2015   

       LeaseMonthStart  LeaseDayStart  LeaseYearEnd  LeaseMonthEnd  \
0                    5             12          2025              6   
1               

In [58]:
df_encoded.to_excel("kontrak-sewa-encoded.xlsx", index=False)

In [72]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6648 entries, 0 to 11369
Data columns (total 60 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   BusinessType                 6648 non-null   float64
 1   BuildingArea                 6648 non-null   float64
 2   CuryUnitPrice                6648 non-null   float64
 3   LeaseYearStart               6648 non-null   int32  
 4   LeaseMonthStart              6648 non-null   int32  
 5   LeaseDayStart                6648 non-null   int32  
 6   LeaseYearEnd                 6648 non-null   int32  
 7   LeaseMonthEnd                6648 non-null   int32  
 8   LeaseDayEnd                  6648 non-null   int32  
 9   LeaseDurationDays            6648 non-null   int64  
 10  LeaseDurationMonths          6648 non-null   int32  
 11  n_subunit                    6648 non-null   int64  
 12  UnitNum                      6648 non-null   object 
 13  ContractType_Casual Le

In [ ]:
target_col = "CuryUnitPrice"

feature_cols = [c for c in df_new.columns if c != target_col]

df_unique = (
    df_new
    .groupby(feature_cols, as_index=False)[target_col]
    .mean()
)

print("Jumlah baris awal :", len(df_new))
print("Jumlah baris unik :", len(df_unique))

Jumlah baris awal : 6652
Jumlah baris unik : 6274


In [86]:
df_unique.to_excel("kontrak-sewa-for-clustering.xlsx", index=False)

In [2]:
import pandas as pd

# misal file disimpan sebagai 'leasing_data.xlsx'
df = pd.read_excel('D:/DATA SKRIPSI/Kontrak-Sewa-Chelsie-250422.xlsx')

# pastikan kolom sesuai dengan file
both_units = (
    df[df['ContractPeriod'].isin(['Daily', 'Monthly'])]
    .groupby('UnitID')['ContractPeriod']
    .nunique()
)
result = both_units[both_units == 2].index.tolist()

print("Unit dengan sewa Daily & Monthly:", result)

Unit dengan sewa Daily & Monthly: ['0LC00002000021', '0LC00002000027', '0LC00K02000016', '0LC00K03000001', '0LC00K03000002', '0LC00K03000004', '0LC00K03000006', '0LC00K03000008', '0LC00K05000001', '0LC00KLG000003', '0LC00KLG000006', '0LC00KUG000001', '0LCATN04000001', '0LCOFF07000011', '0PC00002000048', '0PC00003000021', '0PC00005000099', '0PC000LG000105', '0PC00K02000005', '0PC00K02000007', '0PC00K02000008', '0PC00K02000009', '0PC00K02000016', '0PC00K03000007', '0PC00K03000008', '0PC00K03000011', '0PC00K03000013', '0PC00K03000014', '0PC00K03000015', '0PC00K03000017', '0PC00K04000003', '0PC00K04000009', '0PC00K04000018', '0PC00K04000022', '0PC00K05000005', '0PC00K05000006', '0PC00K05000010', '0PC00K05000017', '0PC00K05000044', '0PC00KUG000001', '0PCATN09000007', '0PCP0101000011', '0PCP0101000061', '0PCP0102000001', '0PCP01LG000006', '0PCP01LG000008', '0PCP01LG000061', '0PCP01UG000004', '0PCP01UG000006', '0PE00001000056', '0PE00003000037', '0PE00K01000005', '0PE00K02000001', '0PE00K0300